<h1><center>Data Science Professional Certificate Final Capstone Project</center></h1>
<h2><center>Daniel Liang</center></h2>
<h3><center>Feb 25, 2020</center></h3>

## 1. Introduction/Business Problem

A private investor expects to open a grocery store or supermarket through joining the Loblaw franchises in Ontario, Canada.  Loblaw is the top grocery store and supermarket franchising company in Canada that offers various franchise banners to be used for the independently owned franchise stores across Canada. Details of Loblaw franchise business opportunities can be found in this [link](https://www.loblaw.ca/en/careers/franchise-opportunities.html). 
  
Available franchise banners from Loblaw that are available for selection include
- [No Frills™](https://www.nofrills.ca)
- [Independent City Market™](https://www.independentcitymarket.ca)
- [Fortinos™](https://www.fortinos.ca)
- [Provigo™](https://www.provigo.ca)
- [Valu-Mart™](https://www.valumart.ca)
- [Your Independent Grocer™](https://www.yourindependentgrocer.ca)  

These banners are different from each other and have their own unique features. Simply put, they are tailored for different types of communities or neighborhoods.  

The private investor wants to open a store at a location where one of the banners from the list could be the best fit. Based on the private investor's description and preference, some other specific criteria should be considered when going through the analysis.
1. The best choice of location is Toronto, but still open to other regions in western Ontario including Kitchener-Waterloo-Cambridge (KWC), London, and Windsor.
2. The pick of location and banner should be optimized to avoid harsh competitions within the same area.  

Through this analysis, multiple appropriate locations with optimized selections of banners should be provided to the private investor to support his decision making. From the result, he should get a clearer and detailed insight of the locations and neighborhood features around them so that he can make the best decision about the business strategy.

## 2. Data

###  2.1 Data Understanding

**The first task** of this project is to figure out what differentiate the banners from each other. One of the straightforward strategies is to analyze the products and brands that each banner provides and find out what types of customer targeted by them. In order to do that, scraping the product and brand list for each banner and spending time on understanding the product-customer relationship is unavoidable. This could be extrodinarily time-consuming, and the return on the investment of time may not be satisfying eventually. However, it can be done from another point of view. Since Loblaw's business has been healthy for many years and its strategy of franchise expanding has been tested working by the real market, we have the confidence to believe that the existing locations of those stores have gone through meticulous determination after tremendous analysis performed by either Loblaw's own business teams or other consulting firms. Therefore, by analyzing the geographical data collected for the neighborhoods around the exsting store locations, we are able to tell the uniqueness of each banner based on the similarities or differences of its neighborhood comparing to others. **The second task** is to analyze multiple regions or cities and try to classify their neighborhoods in such a way that the banners are classified. By doing this, we can propose the top and second choices of banners for each of the neighborhoods in those regions of interest. **The last task**, of course, is to help the private investor avoid potential risk of opening a store with duplicated function as one of the existing stores nearby. Competition with similar banners from other franchise companies than Loblaw is allowable.

#### 2.1.1 Store/Banner Location Data
Running for the first task, we need to have a list of locations for the 6 banners. 

In [1]:
# import modules needed for the report 

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as BS

# import geoby needed for getting coordinates from addresses (not needed)
#import geopy
#from geopy.geocoders import Nominatim

# import pgeocode needed for getting coordinates from postal codes
#!python -m pip install --upgrade pip  # uncomment if needed to upgrade pip package 
#!pip install pgeocode #uncomment to install the pgeocode package

import pgeocode

Locations of stores can usually be found by using the "store locators" provided by each store's website. Taking [No Frills™ store locator](https://www.nofrills.ca/store-locator?searchQuery=Toronto,%20%E5%AE%89%E5%A4%A7%E7%95%A5%E7%9C%81%E5%8A%A0%E6%8B%BF%E5%A4%A7) for example, by inputing the region as Toronto, it shows all the locations with the corresponding addresses and working hours in the list. However, the temporary searched results, as attempted, are not easy to be scraped by Beautifulsoup pakage in the traditional way. Another approach is found and used here, that is, we can use a website scraping software called [Parsehub](https://www.parsehub.com/). It remembers a sequence of selections and clicks needed for searching on the website before scraping the data outputed from the searches. The tutorial can be referred [here](https://www.parsehub.com/blog/how-to-get-the-locations-of-retail-stores-with-web-scraping/).  

From reading some basic descriptions of the banners, it is found that Provigo™ is only used for retailers based in Quebec. Since Quebec does not fall into the private investor's region of interest, Provigo™ is disregarded from our list. After using Parsehub to scrape the locations data of the rest 5 banners from their store locator website, we obtained 5 csv files containing the information we need for geographical data analysis. 

In [2]:
loc_nofrills_url = 'https://raw.githubusercontent.com/liangyiteng888/Data_Science_Pro_Certificate_Capstone/master/nofrills_loc_results.csv'
loc_indcitymarket_url = 'https://raw.githubusercontent.com/liangyiteng888/Data_Science_Pro_Certificate_Capstone/master/IndependentCityMarket_loc_results.csv'
loc_fortinos_url = 'https://raw.githubusercontent.com/liangyiteng888/Data_Science_Pro_Certificate_Capstone/master/fortinos_loc_results.csv'
loc_valumart_url = 'https://raw.githubusercontent.com/liangyiteng888/Data_Science_Pro_Certificate_Capstone/master/valuemart_loc_results.csv'
loc_yourindgrocer_url = 'https://raw.githubusercontent.com/liangyiteng888/Data_Science_Pro_Certificate_Capstone/master/yourindependentgrocer_loc_results.csv'

loc_nofrills = pd.read_csv(loc_nofrills_url)
loc_indcitymarket = pd.read_csv(loc_indcitymarket_url)
loc_fortinos = pd.read_csv(loc_fortinos_url)
loc_valumart = pd.read_csv(loc_valumart_url)
loc_yourindgrocer = pd.read_csv(loc_yourindgrocer_url)

Now, let's take a look at No Frills and Fortinos data we have obtained.

In [3]:
loc_nofrills.head()

,Store_name_Store_name,Store_name_Address,Store_name_Hours,Store_name_In_store
0,ROCCO'S NOFRILLS TORONTO,"200 Front St E, Toronto, Ontario M5A 4T9",Closed Now8:00 AM - 9:00 PM,In-Store Shopping Only
1,MATT'S NOFRILLS TORONTO,"449 Parliament St, Toronto, Ontario M5A 3A3",Closed Now9:00 AM - 9:00 PM,In-Store Shopping Only
2,JOE'S NOFRILLS TORONTO,"345 Bloor St E Unit 1A, Toronto, Ontario M4W 3J6",Closed Now8:00 AM - 9:30 PM,In-Store Shopping Only
3,DAVE & CHARLOTTE'S NOFRILLS TORONTO - GAS BAR,"449 Carlaw Avenue, Toronto, Ontario M4K 3H9",Closed Now8:00 AM - 10:00 PM,In-Store Shopping Only
4,DAVE & CHARLOTTE'S NOFRILLS TORONTO,"449 Carlaw Ave, Toronto, Ontario M4K 3H9",Closed Now8:00 AM - 10:00 PM,NaN


In [4]:
loc_fortinos.head()

,Store_name_Store_name,Store_name_Address,Store_name_Hours,Store_name_In_store
0,NORTH YORK LAWRENCE,"700 Lawrence Ave W, North York, Ontario M6A 3B4",Open Now7:00 AM - 8:00 PM,NaN
1,QUEENS PLATE,"330 Queen's Plate Dr, Etobicoke, Ontario M9W 7J7",Open Now7:00 AM - 8:00 PM,NaN
2,HIGHWAY 7 AND ANSLEY,"3940 Hwy 7 RR 2, Vaughan, Ontario L4L 9C3",Open Now7:00 AM - 8:00 PM,NaN
3,LANGSTAFF,"8585 Hwy 27, RR3, Woodbridge, Ontario L4L 1A7",Open Now7:00 AM - 8:00 PM,NaN
4,MAJOR MACKENZIE,"2911 Major MacKenzie Dr, Vaughan, Ontario L6A 3N9",Open Now7:00 AM - 8:00 PM,NaN


These dataframes have to be cleaned and reorganized for our further usage. First, "hours" and "in-store only" features are not very important in our location analysis, so they will be removed from the table. Then, the column names need to be formatted as "Name" and "Address". Second, the "Address" strings may need to be reworded or separated into different columns if necessary. 

#### 2.1.2 Neighborhood Location Data
Either for analyzing the neiborhoods of the store locations found for the banners or the neighborhoods in the provate investor's regions of interest, we need the geographical data around those points of interest. In this analysis, we will only use location data retrieved using **[Foursquare](https://foursquare.com/)** API. The location data from Foursquare only includes information like restuarants, coffee shops, fitness centers, airports, and so on. This location data is good enough to help figure out what type of life in a neighborhood. As a brief analysis, We will not go very deep into analyzing ages, income level, education, population or other demographics. 

Toronto neighborhoods analysis as well as location data can be carried over from this report ([Part 1 - Data](https://github.com/liangyiteng888/Data_Science_Pro_Certificate_Final_Practice/blob/master/Toronto_Neighborhood_Data_Cleaning.ipynb) and [Part 2 - Analysis](https://github.com/liangyiteng888/Data_Science_Pro_Certificate_Final_Practice/blob/master/Toronto_Neighborhood_Modelling.ipynb)), in which the neighborhoods in Toronto are classified into clusters based on the Foursquare location data. We can use the similiar method to analyze Kitchener-Waterloo-Cambridge (KWC), London, and Windsor. The postal codes within these areas can also be found in this [Wikipedia link](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_N). However, unlike Toronto, all of the postal codes in the western Ontario start with 'N', so we need to extract only the codes for the three regions above from the site. **Beautifulsoap** package for data scraping will be used here to handle this task. 

- **Toronto Neighborhood Location Data (Carried over)**

In [5]:
loc_toronto_url = 'https://raw.githubusercontent.com/liangyiteng888/Data_Science_Pro_Certificate_Final_Practice/master/Toronto_Neighborhood_with_Coordinates.csv'
neighbor_toronto = pd.read_csv(loc_toronto_url, index_col=0)
neighbor_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


- **Kitchener-Waterloo-Cambridge (KWC), London, and Windsor Location Data**

Use BeautifulSoup package to scrape data from the [wiki website](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_N)

In [6]:
data_wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_N').text
soup = BS(data_wiki_url, 'lxml')
#print(soup.prettify())

In [7]:
neighbor_table = soup.find('table',{'cellpadding':'2'})
#print(neighbor_table)

In [8]:
# define a function to extract values from the scraped wikipedia table
def extract_table_data(sample):
    sample = sample.find('span')
    temp_ls=[]
    temp_surfix = None
    
    for cont in sample.contents:
        try:
            try:
                str = cont.contents[0]
            except:
                str = cont.contents
                if str == []:
                    str = ''
            if not (temp_surfix is None):
                str = temp_surfix + ' ' + str
            temp_ls.append((str + ' '))
            temp_surfix = None
#            print(str)
        except:
            if '/' in cont:
                temp_surfix = cont.strip()
            else:
                str = cont
                if not (temp_surfix is None):
                    str = temp_surfix + ' ' + str
                temp_ls.append(str)
                temp_surfix = None
#    print(temp_ls)                      
    content_name = ''.join(temp_ls).strip()
#    print(content_name)   
    return content_name

In [9]:
neighbor_table = neighbor_table.findAll('td')
neighbor_list=[[],[]]
for row in neighbor_table:
    try:
        neighbor_list[0].append(row.find('b').contents[0])
        neighbor_list[1].append(extract_table_data(row))
    except:
        try:
            neighbor_list[1].append(row.find('i').contents[0])
        except:
            print('check the index of \n', row)
            break
#print(neighbor_list)

Convert the scraped data list into a dataframe which has all of the postal codes in western Ontario. We will clean this dataframe and get coordinates of only the postal codes winthin out regions of interest in the next section.

In [10]:
df_neighbor_roi = pd.DataFrame(np.transpose(neighbor_list), columns = ['Postcode','Region'])
df_neighbor_roi.head()

,Postcode,Region
0,N1A,Dunnville
1,N2A,Kitchener East
2,N3A,New Hamburg (Baden )
3,N4A,Not assigned
4,N5A,Stratford North


### 2.2 Data Preparation
For the scraped data, we need to clean the data and also find the coordinates for the store/banner addresses and neighborhoods in our regions of interest.

#### 2.2.1 Getting Coordinates for Store/Banner Addresses
First, let's clean the stores dataset based on the requirements stated in Section 2.1.1. 

In [11]:
# Save all the store dataframes into one dictionary to make it easier to refer to in the later analysis
dict_store_df = {'nofrills':loc_nofrills, 'indcitymarket':loc_indcitymarket, 'fortinos':loc_fortinos, 'valumart':loc_valumart, 'yourindgrocer':loc_yourindgrocer}
dict_store_df_clean = {}
#dir(dict_store_df)
columns = ['Store Name','Address']
for df_name, df_main in dict_store_df.items():
    df_clean = df_main.iloc[:,0:2].drop_duplicates() # remove duplicated rows
    df_clean.reset_index(drop=True, inplace = True) # reset index
#     print(df_clean)  
    df_clean.columns = columns
    df_clean = df_clean[['Store Name','Address']]
    df_clean['Postcode']=[i[-7:] for i in df_clean['Address']]
    df_clean['Address']=[i[:-7] for i in df_clean['Address']]
    dict_store_df_clean[str(df_name)] = df_clean
    
#dict_store_df_clean['valumart'].head()

Create a function to find coordinates based on the Address column and add the coordinates into the dataframe

In [12]:
def coordinates_in_df_V2(df,postcode_col):
    """
    Read in a dataframe and postal codes in the specified column, then return another dataframe that has two additional columns wiht latitudes and longitudes. Will not change the input dataframe.
    """
    pnomi = pgeocode.Nominatim('ca') # country code 'ca'- Canada
    df_temp = df.copy(deep=True)
    df_temp['latitude'] = ''
    df_temp['latitude'] = ''
    for index, row in df_temp.iterrows():
        try:
            location = pnomi.query_postal_code(str(row[postcode_col]))
        except:
            print('Run terminated due to error!')
            print('Check values in row', index)
            print('Contents of the row:\n', row)
            break
        try:
            #print(location.address)
            #print((location.latitude, location.longitude))
            df_temp.loc[index,'latitude'] = location.latitude
            df_temp.loc[index,'longitude'] = location.longitude
        except:
            print('Row', index, "has invalid Address strings that Geolocator doesn't understand and find no coordinates")
            df_temp.loc[index,'latitude'] = np.nan
            df_temp.loc[index,'longitude'] = np.nan
            #break
    return df_temp

In [13]:
dict_store_df_ws_coord = {}
for df_name, df_main in dict_store_df_clean.items():
    print('Working on dataframe ', df_name)
    dict_store_df_ws_coord[df_name] = coordinates_in_df_V2(df_main,'Postcode')

Working on dataframe  nofrills
Working on dataframe  indcitymarket
Working on dataframe  fortinos
Working on dataframe  valumart
Working on dataframe  yourindgrocer


Check 'NaN' values in each of the dataframes and make sure we didn't create any after finding the corresponding coordinates

In [14]:
for df_name, df_main in dict_store_df_ws_coord.items():
    print('Dataframe', df_name, ' has number of "Nan":', df_main.isnull().values.sum())

Dataframe nofrills  has number of "Nan": 0
Dataframe indcitymarket  has number of "Nan": 0
Dataframe fortinos  has number of "Nan": 0
Dataframe valumart  has number of "Nan": 0
Dataframe yourindgrocer  has number of "Nan": 0


Take a look at one of the dataframes, No Frills, for example.

In [15]:
dict_store_df_ws_coord['nofrills'].head()

,Store Name,Address,Postcode,latitude,longitude
0,ROCCO'S NOFRILLS TORONTO,"200 Front St E, Toronto, Ontario",M5A 4T9,43.6555,-79.3626
1,MATT'S NOFRILLS TORONTO,"449 Parliament St, Toronto, Ontario",M5A 3A3,43.6555,-79.3626
2,JOE'S NOFRILLS TORONTO,"345 Bloor St E Unit 1A, Toronto, Ontario",M4W 3J6,43.6827,-79.3730
3,DAVE & CHARLOTTE'S NOFRILLS TORONTO - GAS BAR,"449 Carlaw Avenue, Toronto, Ontario",M4K 3H9,43.6803,-79.3538
4,DAVE & CHARLOTTE'S NOFRILLS TORONTO,"449 Carlaw Ave, Toronto, Ontario",M4K 3H9,43.6803,-79.3538


#### 2.2.2 Getting Coordinates for Neighborhoods in Our Regions of Interest (ROI)

In [16]:
print('The original number of records of the ROI neighborhood data is ', df_neighbor_roi.shape[0])
df_neighbor_roi['Region'].value_counts()[:5] # only shows the top five
#df_neighbor_roi['Region'].values

The original number of records of the ROI neighborhood data is  180


Not assigned                         74
St. Thomas  South                     1
Windsor                               1
Windsor  East (East Walkerville )     1
London  (East Tempo )                 1
Name: Region, dtype: int64

We notice that out of 180 records, 74 have unassigned region. We'll ignore them from our analysis. 

In [17]:
df_neighbor_roi = df_neighbor_roi[df_neighbor_roi['Region'] != 'Not assigned']
print('The number of records of the data after removing "Not assigned" regions is ', df_neighbor_roi.shape[0])

The number of records of the data after removing "Not assigned" regions is  106


Now, let's clean and reorganize the dataframe so that it has a "Borough" column, in which the values will show whether it is Kitchener-Waterloo-Cambridge (KWC), London, or Windsor. At the mean time, the other boroughs not recognized as the ones in our regions of interest will be noted as 'Nan'. The regions we are going to pick should include the following strings:  
- Kitchener as "KWC"
- Waterloo as "KWC"
- Cambridge as "KWC"
- London
- Windsor

In [18]:
df_neighbor_roi['Borough'] = np.nan
for index, row in df_neighbor_roi.iterrows():
    if any([i in row['Region'] for i in ['Kitchener', 'Waterloo', 'Cambridge']]):
        df_neighbor_roi.loc[index,'Borough'] = 'KWC'
        continue
    elif 'London' in row['Region']:
        df_neighbor_roi.loc[index,'Borough'] = 'London'
        continue
    elif 'Windsor' in row['Region']:
        df_neighbor_roi.loc[index,'Borough'] = 'Windsor'
df_neighbor_roi.head(10)

,Postcode,Region,Borough
0,N1A,Dunnville,NaN
1,N2A,Kitchener East,KWC
2,N3A,New Hamburg (Baden ),NaN
4,N5A,Stratford North,NaN
5,N6A,London North (UWO ),London
6,N7A,Goderich,NaN
7,N8A,Wallaceburg,NaN
8,N9A,Windsor (City Centre / NW Walkerville ),Windsor
10,N2B,Kitchener Northeast,KWC
11,N3B,Elmira,NaN


Now, let's remove the "Borough" rows with 'NaN' so that regions not in our interest will be discarded, and reindex the dataframe.

In [19]:
df_neighbor_roi_clean = df_neighbor_roi.dropna()
print('The final number of records is ', df_neighbor_roi_clean.shape[0])
df_neighbor_roi_clean.head()

The final number of records is  56


,Postcode,Region,Borough
1,N2A,Kitchener East,KWC
5,N6A,London North (UWO ),London
8,N9A,Windsor (City Centre / NW Walkerville ),Windsor
10,N2B,Kitchener Northeast,KWC
14,N6B,London Central,London


It's time to find the coordinates for the neighborhoods(postcodes) in our list. The 'pgeocode' package is used here by following this [instruction](https://pypi.org/project/pgeocode/).

In [20]:
df_neighbor_roi_clean = df_neighbor_roi_clean.copy(deep=True) # reset the memory locatoin of the dataframe to avoid the copy warning message
df_neighbor_roi_clean['Latitude']=''
df_neighbor_roi_clean['Longitude']=''
pnomi = pgeocode.Nominatim('ca') # country code 'ca'- Canada
for index, row in df_neighbor_roi_clean.iterrows():   
    geoloc=pnomi.query_postal_code(str(row['Postcode'])) 
    #print(geoloc['latitude']:'longitude'])
    df_neighbor_roi_clean.loc[index,'Latitude'] = geoloc['latitude']
    df_neighbor_roi_clean.loc[index,'Longitude'] = geoloc['longitude']
df_neighbor_roi_clean.head()

,Postcode,Region,Borough,Latitude,Longitude
1,N2A,Kitchener East,KWC,43.4413,-80.4246
5,N6A,London North (UWO ),London,42.9976,-81.2563
8,N9A,Windsor (City Centre / NW Walkerville ),Windsor,42.2007,-83.0276
10,N2B,Kitchener Northeast,KWC,43.4646,-80.4467
14,N6B,London Central,London,42.9835,-81.2386


We can group our data so that rows with the same Borough stay together, and let's also put the Borough column ahead of the Region column. Notice the difference from the Toronto neighborhood data that the data collected for these regions of interest do not have detailed information like 'Borough' or 'Neighborhood' given by the website directly. We will just call the Regions as 'Neighborhood' from now on.

In [21]:
df_neighbor_roi_grouped = df_neighbor_roi_clean[['Postcode', 'Borough', 'Region']+df_neighbor_roi_clean.columns.to_list()[-2:]]
df_neighbor_roi_grouped.rename(columns={'Region':'Neighborhood'},inplace=True)
df_neighbor_roi_grouped = df_neighbor_roi_grouped.sort_values(by = 'Borough').reset_index(drop=True)
print('Check the number of records is ', df_neighbor_roi_grouped.shape[0])
df_neighbor_roi_grouped

Check the number of records is  56


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,N2A,KWC,Kitchener East,43.4413,-80.4246
1,N2V,KWC,Waterloo Northwest,43.4764,-80.5842
2,N2T,KWC,Waterloo Southwest,43.453,-80.5692
3,N1T,KWC,Cambridge East,43.3849,-80.2833
4,N1S,KWC,Cambridge Southwest,43.3592,-80.3347
5,N2R,KWC,Kitchener South,43.3849,-80.487
6,N1R,KWC,Cambridge Central,43.3666,-80.2239
7,N2P,KWC,Kitchener Southeast,43.3878,-80.4296
8,N1P,KWC,Cambridge South,43.3398,-80.2955
9,N2N,KWC,Kitchener West,43.426,-80.5438


Let's save the cleaned data into a csv file for later analysis. In the next section, this coordinates data will be used through Foursquare API to retrieve location data for exploratory analysis. 

In [22]:
df_neighbor_roi_grouped.to_csv('roi_neighbor_cleaned.csv')